# Compute MHWs from moored in situ temperature


The goal of the code is to compute Marine Heatwaves (MHWs) from daily temperature time-series data obtained from the ORS065 mooring. It processes the data by selecting the input files, gridding the mooring data onto a regular grid (still daily, but with specific start and date), and then applying the MHW detection algorithm. The detected MHW events are analyzed and relevant information is saved in a shelf file for further analysis. The code iterates over multiple files, each representing a different depth, to perform the analysis and save the results.

In [11]:
import numpy as np
import datetime
import glob
import shelve
import marineHeatWaves_AS_v2 as mhw_v2
from matplotlib import pyplot as plt
from datetime import date
import pandas as pd

In [2]:
# Dataset
list_FILES = sorted(glob.glob('./DATA_raw_mooring_ORS065/file_ORS065*_NEWdepths.csv'))
N_FILES = len(list_FILES)
print(N_FILES)

ClimatologyPeriod = [1992, 2019]
MHWPeriod = [1992, 2019]
mhwname = 'MHWS_2020'

# Create a regular daily time vector from 1 Jan 1992 to 31 Dec 2019
times_dates = np.arange(
    datetime.datetime(ClimatologyPeriod[0], 1, 1),
    datetime.datetime(ClimatologyPeriod[1] + 1, 1, 1),
    datetime.timedelta(days=1)
).astype(datetime.datetime)
t = np.array([times_dates[i].toordinal() for i in range(len(times_dates))])




12


In [3]:
# Function to read the mooring data
def fun_getdata(FILE):
    my_data_str = np.genfromtxt(FILE, delimiter=',', dtype='str')
    depth = int(my_data_str[1, 0])
    T = len(my_data_str) - 1  # 0 is the title
    format = "%Y-%m-%d"

    times_dates0 = np.array([datetime.datetime.strptime(my_data_str[i, 1], format) for i in range(1, T)]) # 0 is the title
    t0 = np.array([times_dates0[i].toordinal() for i in range(len(times_dates0))])
    sst0 = np.array([float(my_data_str[i, 2]) for i in range(1, T)]) # 0 is the title
        
    return depth, t0, times_dates0, sst0

# Function to match the mooring data with a regular grid
def fun_interp(t0, sst0, t):
    # Interpolate onto regular grid
    sst = np.zeros(len(t)) + np.nan
    t0_day = np.int0(t0)
    for i in range(len(t)):
        # Match day-of-year values and average if the resolution is higher than daily
        t0_ind = np.where(t0_day == t[i])[0]
        if np.isfinite(t0_ind):
            sst[i] = np.nanmean(sst0[t0_ind.astype(int)])
     
    # Calculate the percentage of days with NaN values
    Nb_nans = np.sum(np.isnan(sst)) / len(sst)
    print('Percentage of NaN= ' + str(Nb_nans))
    
    return times_dates, sst, Nb_nans

# Function to create a time-series with the MHW information
def fun_get_ano_timeseries(sst, clim, mhws):
    sst_ano_thresh = sst - clim['thresh']
    sst_ano = sst - clim['seas']

    # More time-series
    sst_ano_pos = sst * 0  # only for positive anomalies
    list_MHWstats_1_0 = sst * 0  # list of 0, same size as list_sst

    # Fill values only when MHW
    for i in range(mhws['n_events']):
        ind_MHW = np.array(range(mhws['index_start'][i], mhws['index_end'][i] + 1))
        list_MHWstats_1_0[ind_MHW] = 1
        sst_ano_pos[ind_MHW] = sst_ano[ind_MHW]  # only for positive anomalies

    # Add data to the clim dictionary
    clim['sst'] = sst
    clim['times_dates'] = times_dates
    clim['t'] = t
    clim['list_MHWstats_1_0'] = list_MHWstats_1_0
    clim['sst_ano_thresh'] = sst_ano_thresh
    clim['sst_ano'] = sst_ano
    
    return sst_ano, sst_ano_thresh, sst_ano_pos, sst_ano_pos, list_MHWstats_1_0

# Function to save the data in a shelf
def fun_save_shelf(mhws, clim, dict_year_stats, depth):
    depth_order = '0' + str(depth)
    with shelve.open('DATA_processed/SAVE_ORS065_mhws_Strength2018' + '_z' + str(depth_order[-2:])) as d:
        d['dict_mhws'] = mhws
        d['dict_clim'] = clim
        d['dict_year_stats'] = dict_year_stats
        d['DEPTHS'] = depth

        
        

In [7]:
# Function to match the mooring data with a regular grid
def fun_stats_years_months_days_WinterSummer(t,mhws,depth,list_MHWstats_1_0,dict_year_stats):
    colours = (plt.rcParams['axes.prop_cycle'].by_key()['color'])
    t_all_MHWdays = t[np.where(list_MHWstats_1_0==1)]
    
    # Create dataframe
    df_ORS_all_daily=pd.DataFrame(list_MHWstats_1_0,pd.DatetimeIndex(times_dates, name='t'))
    df_ORS_all_daily.columns = ['MHWdays']

    ### Seasons 3 months
    winter_3m = ((df_ORS_all_daily.index.month >6) & (df_ORS_all_daily.index.month <10)) # June - Aug
    summer_3m = ((df_ORS_all_daily.index.month <4)) # Jan - Mar # 


    df_ORS_all_yearly_MHWdays_year = df_ORS_all_daily.groupby(df_ORS_all_daily.index.year).sum()
    df_ORS_all_yearly_MHWdays_winter_3m = df_ORS_all_daily[winter_3m].groupby(df_ORS_all_daily[winter_3m].index.year).sum()
    df_ORS_all_yearly_MHWdays_summer_3m = df_ORS_all_daily[summer_3m].groupby(df_ORS_all_daily[summer_3m].index.year).sum()
    
    dict_year_stats['n_days_years_winter2_3m'] = np.array(df_ORS_all_yearly_MHWdays_winter_3m['MHWdays'])
    dict_year_stats['n_days_years_summer2_3m'] = np.array(df_ORS_all_yearly_MHWdays_summer_3m['MHWdays'])

    dict_year_stats['t'] = t
    dict_year_stats['years'] = np.array(df_ORS_all_yearly_MHWdays_year.index  )
    dict_year_stats['n_days_years'] = np.array(df_ORS_all_yearly_MHWdays_year['MHWdays'])


In [12]:
# Loop over each file
for f in range(N_FILES):
    FILE = list_FILES[f]
    print(FILE)

    dict_year_stats = {} 
    depth, t0, times_dates0, sst0 = fun_getdata(FILE)
    times_dates, sst, Nb_nans = fun_interp(t0, sst0, t)
    dict_year_stats = {}
    
    ## Run MHW algorithm from Hobday et al, 2016 and https://github.com/ecjoliver/marineHeatWaves  
    mhws, clim = mhw_v2.detect(t, sst, climatologyPeriod=ClimatologyPeriod, MHWPeriod=MHWPeriod, smoothPercentileWidth=31)
    sst[clim['missing']] = np.nan  # reverse the padding of sst

    # Re-format into time-series
    sst_ano, sst_ano_thresh, sst_ano_pos, sst_ano_pos, list_MHWstats_1_0 = fun_get_ano_timeseries(sst, clim, mhws)

    fun_stats_years_months_days_WinterSummer(t,mhws,depth,list_MHWstats_1_0,dict_year_stats)

    
    # Save data
    fun_save_shelf(mhws, clim, dict_year_stats, depth)

./DATA_raw_mooring_ORS065\file_ORS065_temp_Z001_NEWdepths.csv


C:\Users\Amandine\AppData\Local\Temp\ipykernel_10616\2534439008.py:18: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  t0_day = np.int0(t0)
C:\Users\Amandine\AppData\Local\Temp\ipykernel_10616\2534439008.py:23: RuntimeWarning: Mean of empty slice
  sst[i] = np.nanmean(sst0[t0_ind.astype(int)])


Percentage of NaN= 0.6167986701867605
./DATA_raw_mooring_ORS065\file_ORS065_temp_Z005_NEWdepths.csv
Percentage of NaN= 0.6167986701867605
./DATA_raw_mooring_ORS065\file_ORS065_temp_Z010_NEWdepths.csv
Percentage of NaN= 0.6167008898015058
./DATA_raw_mooring_ORS065\file_ORS065_temp_Z015_NEWdepths.csv
Percentage of NaN= 0.28297643492715363
./DATA_raw_mooring_ORS065\file_ORS065_temp_Z020_NEWdepths.csv
Percentage of NaN= 0.16828004302336952
./DATA_raw_mooring_ORS065\file_ORS065_temp_Z025_NEWdepths.csv
Percentage of NaN= 0.17287572113034125
./DATA_raw_mooring_ORS065\file_ORS065_temp_Z030_NEWdepths.csv
Percentage of NaN= 0.16818226263811478
./DATA_raw_mooring_ORS065\file_ORS065_temp_Z035_NEWdepths.csv
Percentage of NaN= 0.16593331377725629
./DATA_raw_mooring_ORS065\file_ORS065_temp_Z040_NEWdepths.csv
Percentage of NaN= 0.16642221570352989
./DATA_raw_mooring_ORS065\file_ORS065_temp_Z045_NEWdepths.csv
Percentage of NaN= 0.16837782340862423
./DATA_raw_mooring_ORS065\file_ORS065_temp_Z050_NEWdept